In [ ]:
import os, json, requests
from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel, Field

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print("client_ready=", bool(client))


In [ ]:
# Define tool and messages

def get_weather(latitude: float, longitude: float):
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    return response.json()["current"]

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current temperature for provided coordinates in celsius.",
            "parameters": {
                "type": "object",
                "properties": {
                    "latitude": {"type": "number"},
                    "longitude": {"type": "number"},
                },
                "required": ["latitude", "longitude"],
                "additionalProperties": False,
            },
            "strict": True,
        },
    }
]

system_prompt = "You are a helpful weather assistant."
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": "What's the weather like in Paris today?"},
]


In [ ]:
# Step 1: Call model with tool and inspect message + tool_call ids
from IPython.display import display, JSON

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

print("response_id:", completion.id)
assistant_msg = completion.choices[0].message

display(JSON(assistant_msg.model_dump(), expanded=False))

tool_calls = assistant_msg.tool_calls or []
messages.append(assistant_msg)
print("num_tool_calls:", len(tool_calls))


In [ ]:
# Step 2: Execute tool(s) and show each call's id
from IPython.display import display, JSON

results = []
for tc in tool_calls:
    name = tc.function.name
    args = json.loads(tc.function.arguments)
    if name == "get_weather":
        result = get_weather(**args)
    else:
        result = {"error": f"unknown function {name}"}
    results.append({"id": tc.id, "name": name, "args": args, "result": result})
    messages.append({"role": "tool", "tool_call_id": tc.id, "content": json.dumps(result)})

display(JSON(results, expanded=False))


In [ ]:
# Step 3: Structured output parse and display
class WeatherResponse(BaseModel):
    temperature: float = Field(description="The current temperature in celsius for the given location.")
    response: str = Field(description="A natural language response to the user's question.")

completion_2 = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=messages,
    tools=tools,
    response_format=WeatherResponse,
)

final_response = completion_2.choices[0].message.parsed
print({
    "temperature": final_response.temperature,
    "response": final_response.response,
})
